In [ ]:
# \/ pypi: `pip install aiofiles protobuf grpcio`
import grpc

# \/ nvidia gitlab: https://gitlab-master.nvidia.com/alpamayo/alpasim-grpc-api/
from alpasim_grpc.v0.egodriver_pb2_grpc import EgodriverServiceStub
from alpasim_utils.logs import async_read_pb_log

## Show an example of how a log can be parsed and all messages sent to the alpamodel be extracted

In [4]:
log_path = "<insert/path/to/valid/asl/file.asl>"

async for message in async_read_pb_log(log_path):
    if "driver" not in message.WhichOneof("log_entry"):
        continue

    print(message.WhichOneof("log_entry"))

driver_session_request
driver_ego_trajectory
driver_camera_image
driver_camera_image
driver_camera_image
driver_request
driver_return
driver_ego_trajectory
driver_camera_image
driver_camera_image
driver_camera_image
driver_request
driver_return
driver_ego_trajectory
driver_camera_image
driver_camera_image
driver_camera_image
driver_request
driver_return
driver_ego_trajectory
driver_camera_image
driver_camera_image
driver_request
driver_return
driver_ego_trajectory
driver_camera_image
driver_camera_image
driver_request
driver_return
driver_ego_trajectory
driver_camera_image
driver_camera_image
driver_request
driver_return
driver_ego_trajectory
driver_camera_image
driver_camera_image
driver_request
driver_return
driver_ego_trajectory
driver_camera_image
driver_camera_image
driver_request
driver_return
driver_ego_trajectory
driver_camera_image
driver_camera_image
driver_request
driver_return
driver_ego_trajectory
driver_camera_image
driver_camera_image
driver_request
driver_return
driver_

## Replay the log to alpamodel
Assumes that the alpamodel gRPC API is up and running on `localhost:6500`. We will parse the log and re-send to the API message by message.

Note that if the simulation was done with batch size >1 it would be more tricky than this because a single `.asl` log contains only one rollout whereas the original simulation contained more.

In [6]:
async with grpc.aio.insecure_channel("localhost:6500") as channel:
    egodriver_stub = EgodriverServiceStub(channel)

    async for message in async_read_pb_log(log_path):
        entry_kind = message.WhichOneof("log_entry")
        if "driver" not in entry_kind:
            continue

        match entry_kind:
            case "driver_session_request":
                _ = await egodriver_stub.start_session(message.driver_session_request)
            case "driver_ego_trajectory":
                _ = await egodriver_stub.submit_egomotion_observation(message.driver_ego_trajectory)
            case "driver_camera_image":
                _ = await egodriver_stub.submit_image_observation(message.driver_camera_image)
            case "driver_request":
                api_response = await egodriver_stub.drive(message.driver_request) # new response
            case "driver_return":
                logged_response = message.driver_return # old response, can compare